<a href="https://colab.research.google.com/github/laserine32/serba-sd/blob/main/py_diffuser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1️⃣ : Install

In [ ]:
#@title ## <img src="https://ssl.gstatic.com/images/branding/product/1x/drive_2020q4_48dp.png" height="20px"> Mount GDrive
#@markdown Is used to store generation results. the results of the generation will be saved on `SD-IMG-OUT/img`
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
clear_output()
flag_gfpgan_installed = False
print("✅ Done!")

In [ ]:
#@title ## ⚙️ Install
#@markdown Install Dependcy
!pip install xformers transformers omegaconf safetensors accelerate compel

!pip install diffusers
!pip install scipy ftfy


!wget https://huggingface.co/runwayml/stable-diffusion-v1-5/raw/main/v1-inference.yaml
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py

clear_output()
print("✅ Done!")

In [ ]:
#@title ## 🏰 Install GFPGAN
#@markdown used for facial restoration and enhancement. If you don't want to install you can skip it and install later if you need it.
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions

!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.1.0/GFPGANv1.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -P experiments/pretrained_models

%cd /content
clear_output()
flag_gfpgan_installed = True
print("✅ Done!")

In [ ]:
#@title ## ⚙️ Init
#@markdown initialize the program defining pipes and functions
import diffusers
import transformers

import sys
import os
import shutil
import time

import torch
import matplotlib.pyplot as plt
import numpy as np

from diffusers import StableDiffusionPipeline

from PIL import Image
import hashlib
import datetime
import random

if torch.cuda.is_available():
    device_name = torch.device("cuda")
    torch_dtype = torch.float16
else:
    device_name = torch.device("cpu")
    torch_dtype = torch.float32

# Plot pipeline outputs.
def plot_images(images, labels = None):
    N = len(images)
    n_cols = 5
    n_rows = int(np.ceil(N / n_cols))

    plt.figure(figsize = (20, 5 * n_rows))
    for i in range(len(images)):
        plt.subplot(n_rows, n_cols, i + 1)
        if labels is not None:
            plt.title(labels[i])
        plt.imshow(np.array(images[i]))
        plt.axis(False)
    plt.show()

def get_row_col(data):
  col = 6
  row = 1
  if len(data) > 6:
    len_data = len(data)
    row = 2
    if len_data > 12:
      row = len_data // col + 1
    if len_data % col == 0:
      row = len_data // col
  return col, row

def image_grid(imgs):
    cols, rows = get_row_col(imgs)
    w, h = imgs[0].size
    aspect_ratio = h / w
    w = 256
    h = int(w * aspect_ratio)
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        hmi = img.resize((w, h))
        grid.paste(hmi, box=(i%cols*w, i//cols*h))
    return grid

# Prompt embeddings to overcome CLIP 77 token limit.
# https://github.com/huggingface/diffusers/issues/2136

def get_prompt_embeddings(
    pipe,
    prompt,
    negative_prompt,
    split_character = ",",
    device = torch.device("cpu")
):
    max_length = pipe.tokenizer.model_max_length
    # Simple method of checking if the prompt is longer than the negative
    # prompt - split the input strings using `split_character`.
    count_prompt = len(prompt.split(split_character))
    count_negative_prompt = len(negative_prompt.split(split_character))

    # If prompt is longer than negative prompt.
    if count_prompt >= count_negative_prompt:
        input_ids = pipe.tokenizer(
            prompt, return_tensors = "pt", truncation = False
        ).input_ids.to(device)
        shape_max_length = input_ids.shape[-1]
        negative_ids = pipe.tokenizer(
            negative_prompt,
            truncation = False,
            padding = "max_length",
            max_length = shape_max_length,
            return_tensors = "pt"
        ).input_ids.to(device)

    # If negative prompt is longer than prompt.
    else:
        negative_ids = pipe.tokenizer(
            negative_prompt, return_tensors = "pt", truncation = False
        ).input_ids.to(device)
        shape_max_length = negative_ids.shape[-1]
        input_ids = pipe.tokenizer(
            prompt,
            return_tensors = "pt",
            truncation = False,
            padding = "max_length",
            max_length = shape_max_length
        ).input_ids.to(device)

    # Concatenate the individual prompt embeddings.
    concat_embeds = []
    neg_embeds = []
    for i in range(0, shape_max_length, max_length):
        concat_embeds.append(
            pipe.text_encoder(input_ids[:, i: i + max_length])[0]
        )
        neg_embeds.append(
            pipe.text_encoder(negative_ids[:, i: i + max_length])[0]
        )

    return torch.cat(concat_embeds, dim = 1), torch.cat(neg_embeds, dim = 1)

def restore_face(out_dir, fn, model_version, scl):
  source_img = "/content/GFPGAN/tmp_img/"+fn+".png"
  tmp_out = "/content/GFPGAN/tmp_out/"
  os.makedirs(tmp_out, exist_ok=True)
  !python /content/GFPGAN/inference_gfpgan.py -i $source_img -o $tmp_out -v $model_version -s $scl --bg_upsampler realesrgan
  frm_img = f"{tmp_out}restored_imgs/{fn}.png"
  !mv $frm_img $out_dir
  return Image.open(out_dir+"/"+fn+".png")


clear_output()
print("✅ Done!")

# Step 2️⃣ : 📥 Download Model

In [ ]:
#@title ## From : 🤗 Huggingface

#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/runwayml/stable-diffusion-v1-5), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
# https://huggingface.co/settings/tokens
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

#@markdown #### <a name="diffusionmodel"><font size="4" color="#e8cf53">**Diffusion Model**:</font></a>
MODEL_ID = 'runwayml/stable-diffusion-v1-5' #@param ["runwayml/stable-diffusion-v1-5", "runwayml/stable-diffusion-inpainting", "CompVis/stable-diffusion-v1-4-original", "CompVis/stable-diffusion-v1-3","CompVis/stable-diffusion-v1-2","CompVis/stable-diffusion-v1-1","SG161222/Realistic_Vision_V5.1_noVAE","hakurei/waifu-diffusion","nitrosocke/redshift-diffusion","lambdalabs/sd-pokemon-diffusers","doohickey/trinart-waifu-diffusion-50-50","spav/nilou-waifu-diffusion","valhalla/sd-wikiart-v2","rrustom/stable-architecture-diffusers","AstraliteHeart/pony-diffusion","nitrosocke/redshift-diffusion","nitrosocke/Arcane-Diffusion","prompthero/openjourney", "nitrosocke/Nitro-Diffusion", ""]{allow-input: true}
# markdown <font size="3">Use the drop-down arrow to select a model, or enter in a custom model from Hugging Face.</font><br>
# markdown <font size="3">The model `nitrosocke/redshift-diffusion` does not support `LOW_VRAM_PATCH` mode.</font><br>
# markdown <font size="3">If using `CACHE_PIPELINES` you will need to run `RECACHE_PIPES` once when switching `MODEL_ID`. Allows custom input (for HF models not listed)</font>

clear_output()
print("✅ Done!")

In [ ]:
#@title ## From : 🧩 Other
#@markdown #### Select only one of the methods below to download the model
#@markdown #### <font color="#FFFF00">Note: this program can only run **one model per runtime**, so if you want to change the model you have to **restart the runtime**.
from IPython.display import clear_output
import os
import sys
import requests
from urllib.parse import unquote
import time
import threading
# from clint.textui import progress

model_filename = ""

def humansize_speed(nbytes):
    suffixes = ['bps', 'Kbps', 'Mbps', 'Gbps', 'Tbps', 'Pbps']
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

def humansize(nbytes):
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

def format_duration(duration):
    mapping = [
        ('s', 60),
        ('m', 60),
        ('h', 24),
    ]
    duration = int(duration)
    result = []
    for symbol, max_amount in mapping:
        amount = duration % max_amount
        result.append(f'{amount}{symbol}')
        duration //= max_amount
        if duration == 0:
            break

    if duration:
        result.append(f'{duration}d')

    return ' '.join(reversed(result))

def download_model(DirectLink_URL):
    if not DirectLink_URL:
        print("Skipping download as no valid Type or DirectLink_URL provided.")
        return None

    output_path = "/content/custom/Stable-diffusion/"
    # /content/ui/embeddings

    # Create the directories if they don't exist
    os.makedirs(output_path, exist_ok=True)

    urls = [url.strip() for url in DirectLink_URL.split(",")]
    # download_asinkron(urls, output_path)
    url = ""
    if "drive.google.com" in url:
      print("Doesn't support yet!")
      return None
    else:
      url = DirectLink_URL

    print("[GET] "+url)
    response = requests.get(url, stream=True)
    if response.status_code == 200:
      content_disposition = response.headers.get('content-disposition')
      total_length = response.headers.get('content-length')
      if content_disposition:
        filename = unquote(content_disposition.split('filename=')[1])
      else:
        filename = unquote(url.split("/")[-1])  # Extracting the filename from the URL
      # Remove double quotes and semicolons from the filename
      filename = filename.replace('"', '').replace(';', '')
      filename = os.path.join(output_path, filename)  # Modify the filename to include the output path

      if not os.path.exists(filename):
        chunk_size = 5242880  # 5 MB
        with open(filename, 'wb') as f:
          dl = 0
          start = time.perf_counter()
          total_length = int(total_length)
          for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:
              dl += len(chunk)
              f.write(chunk)
              done = int(50 * dl / total_length)
              tmm = format_duration(time.perf_counter() - start)
              kecepatan = humansize_speed(dl//(time.perf_counter() - start))
              progres = humansize(dl)+" / "+humansize(total_length)
              sys.stdout.write("\r[%s%s] %s | %s | %s" % ('=' * done, ' ' * (50-done), progres, kecepatan, tmm))
              sys.stdout.flush()
        print("")
        print("✅ Done", "success", filename)
        return filename
      else:
        print("File already exists:", filename)
        return filename

def download_drive(dl, pt):
  if dl == "":
    print("Drive link empty")
    return False
  if pt != "":
    %cd "$pt"
  ld = "https://drive.google.com/uc?id="+dl.split("/")[5]
  print("Downloading file: "+ld)
  !gdown -q $ld
  print("✅ Done", "success", dl)
  %cd "/content/"
  return True

def load_model_fromfile(path_name):
  get_fname = path_name.split("/")[-1]
  get_ext = get_fname.split(".")[-1]
  dump_path = "/content/custom/"+get_fname.split(".")[0]
  safetensors_param = "--from_safetensors" if get_ext=="safetensors" else ""
  if os.path.exists(dump_path):
    print("DONE")
    return dump_path
  !python convert_original_stable_diffusion_to_diffusers.py \
    --checkpoint_path $path_name \
    --dump_path $dump_path \
    --original_config_file /content/v1-inference.yaml \
    $safetensors_param
  return dump_path

opp = "/content/custom/Stable-diffusion/"
os.makedirs(opp, exist_ok=True)

MODEL_ID = ""
mod_name = None
#@markdown ---
#@markdown # 1. Direct Download
#@markdown If you use a model provider such as [Civitai](https://civitai.com/), [Tensor.Art](https://tensor.art/), etc., then copy the direct download link then paste it
# Call the function with the first set of parameters
DirectLink_URL = ""  # @param {'type': 'string'}
if DirectLink_URL != "":
  mod_name = download_model(DirectLink_URL)


#@markdown ---
#@markdown # 2. GDrive Download
#@markdown If you want to download a model from another gdrive, then apart from inputting the `link` you also have to input the `file name`.
GDRIVE_LINK = "" #@param {type:"string"}
FILE_NAME = "" #@param {type:"string"}
if GDRIVE_LINK != "" and FILE_NAME != "":
  if download_drive(GDRIVE_LINK, opp):
    mod_name = os.path.join(opp, FILE_NAME)
#@markdown ---
#@markdown # 3. Local GDrive
#@markdown load model from local gdrive.
GDRIVE_PATH = "" #@param {type:"string"}
if GDRIVE_PATH != "":
  mod_name = GDRIVE_PATH

#@markdown ---

if mod_name != None:
  MODEL_ID = load_model_fromfile(mod_name)
  # clear_output()
  print(MODEL_ID)
  print("✅ Done!")
else:
  print("❌ Something went wrong!")

In [ ]:
#@title ## LoRA Downloader

def download_lora(Type, DirectLink_URL):
    if Type == "None" or not DirectLink_URL:
        print("Skipping download as no valid Type or DirectLink_URL provided.")
        return

    output_path = "/content/custom/"
    # /content/ui/embeddings
    if Type == "LoRA":
        output_path += "Lora/"
    else:
        print("Invalid type specified.")
        return

    # Create the directories if they don't exist
    os.makedirs(output_path, exist_ok=True)

    urls = [url.strip() for url in DirectLink_URL.split(",")]
    download_asinkron(urls, output_path)
    # if Type == "Checkpoint":
      # download_asinkron(urls, output_path)
    # else:
      # download_sinkron(urls, output_path)

def download_asinkron(urls, output_path):
  tali = []
  print("-- Create Thread")
  for url in urls:
    if "drive.google.com" in url:
      t = threading.Thread(target=download_drive, args=(url,output_path))
    else:
      t = threading.Thread(target=download_response, args=(url,output_path))
    tali.append(t)
  print("-- Start Thread")
  for t in tali:
    t.start()
  for t in tali:
    t.join()

  print("✅✅✅✅✅✅✅✅✅✅")
  print("-- All Download Done! --")
  print("✅✅✅✅✅✅✅✅✅✅")

def download_sinkron(urls, output_path):
  for url in urls:
    if "drive.google.com" in url:
      download_drive(url, output_path)
    else:
      download_response(url, output_path)

      # break

def download_response(url, output_path):
    if url!="":
        print("[GET] "+url)
        response = requests.get(url, stream=True)

        if response.status_code == 200:
            content_disposition = response.headers.get('content-disposition')
            total_length = response.headers.get('content-length')
            if content_disposition:
                filename = unquote(content_disposition.split('filename=')[1])
            else:
                filename = unquote(url.split("/")[-1])  # Extracting the filename from the URL

            # Remove double quotes and semicolons from the filename
            filename = filename.replace('"', '').replace(';', '')

            filename = os.path.join(output_path, filename)  # Modify the filename to include the output path

            if not os.path.exists(filename):
                # print("Downloading file:", filename)
                chunk_size = 5242880  # 5 MB
                with open(filename, 'wb') as f:
                    dl = 0
                    start = time.perf_counter()
                    total_length = int(total_length)
                    for chunk in response.iter_content(chunk_size=chunk_size):
                    # for chunk in progress.bar(response.iter_content(chunk_size=chunk_size), expected_size=(total_length/1024) + 1):
                        if chunk:
                            dl += len(chunk)
                            f.write(chunk)
                            done = int(50 * dl / total_length)
                            tmm = format_duration(time.perf_counter() - start)
                            kecepatan = humansize_speed(dl//(time.perf_counter() - start))
                            progres = humansize(dl)+" / "+humansize(total_length)
                            # sys.stdout.write("\r[%s%s] %s | %s | %s" % ('=' * done, ' ' * (50-done), progres, kecepatan, tmm))
                            # sys.stdout.flush()
                            # f.flush()

                # print("File downloaded successfully.")
                print("✅ Done", "success", filename)
            else:
                print("File already exists:", filename)
        else:
            print("Failed to download the file:", url)

    # Clear the output to keep the notebook clean
    # clear_output()

    # Print the success message

#@markdown ---
#@markdown Downloader
#@markdown ---
# Call the function with the first set of parameters
Type1 = "LoRA"
DirectLink_URL = ""  # @param {'type': 'string'}
if Type1 != "None":
  print("========================================================")
  print("Download "+Type1)
  print("========================================================")
  download_lora(Type1, DirectLink_URL)
  print("========================================================")

clear_output()
print("✅ Done!")

✅ Done!


# Step 3️⃣ : Run

In [ ]:
#@markdown Run this for first time running or change setting.
from diffusers import DPMSolverMultistepScheduler

# Clip skip = 2.
# https://github.com/huggingface/diffusers/issues/3212

clip_skip = 1 #@param {type:"slider", min:1, max:2, step:1}

if clip_skip > 1:
    text_encoder = transformers.CLIPTextModel.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        subfolder = "text_encoder",
        num_hidden_layers = 12 - (clip_skip - 1),
        torch_dtype = torch_dtype
    )
if clip_skip > 1:
    pipe = diffusers.DiffusionPipeline.from_pretrained(
        MODEL_ID,
        torch_dtype = torch_dtype,
        safety_checker = None,
        text_encoder = text_encoder,
    )
else:
    pipe = diffusers.DiffusionPipeline.from_pretrained(
        MODEL_ID,
        torch_dtype = torch_dtype,
        safety_checker = None
    )
pipe = pipe.to(device_name)


scheduler = "Euler A" # @param ["Euler A", "Euler", "DPM", "LMS", "DDIM", "PNDM", "DDPM"]
if scheduler == "Euler A":
  pipe.scheduler = diffusers.EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
elif scheduler == "DPM":
  pipe.scheduler = diffusers.DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
elif scheduler == "LMS":
  pipe.scheduler = diffusers.LMSDiscreteScheduler.from_config(pipe.scheduler.config)
elif scheduler == "DDIM":
  pipe.scheduler = diffusers.DDIMScheduler.from_config(pipe.scheduler.config)
elif scheduler == "Euler":
  pipe.scheduler = diffusers.EulerDiscreteScheduler.from_config(pipe.scheduler.config)
elif scheduler == "PNDM":
  pipe.scheduler = diffusers.PNDMScheduler.from_config(pipe.scheduler.config)
else: # DDPM
  pipe.scheduler = diffusers.DDPMScheduler.from_config(pipe.scheduler.config)

IS_LORA = False


In [ ]:

#@markdown ---


# MODEL_ID = "/content/custom/realisticVisionV51_v51VAE"
LORA_PATH = "" #@param {type:"string"}

if LORA_PATH!= "":
  if not IS_LORA:
    pipe.load_lora_weights(LORA_PATH)
    IS_LORA = True
else:
  if IS_LORA:
    pipe.unload_lora_weights()
    IS_LORA = False
#@markdown - <font size="2" color="#5bc0de">if you using LoRA Downloader the file will be in `custom/LoRA`. Right click the LoRA file then **copy path** and then paste it.</font>


use_prompt_embeddings = True #@param {type:"boolean"}
#@markdown - <font size="2" color="#5bc0de">Used for preprocessing prompts.</font>

prompt = "" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
#@markdown - <font size="2" color="#5bc0de">You can't use (thing:1.2) (other thing:0.7) in diffusers. Is not exactly equals, but you use thing++ other thing--- instead.</font>


prompt_embeds, negative_prompt_embeds = get_prompt_embeddings(
    pipe,
    prompt,
    negative_prompt,
    split_character = ",",
    device = device_name
)

num_inference_steps = 30 #@param {type:"slider", min:10, max:150, step:1}
guidance_scale = 7 #@param {type:"slider", min:1, max:30, step:0.5}
seeds =  -1 #@param {type:"integer"}
#@markdown - <font size="2" color="#5bc0de">Seed for image generation, leave -1 to use a random seed</font>
batch_size = 1 #@param {type:"integer"}
batch_seed = -1  #@param {type:"integer"}
#@markdown - <font size="2" color="#5bc0de">Seed for batch image generation, leave -1 to use a random seed for every generated images.</font>
#@markdown - <font size="2" color="#5bc0de">If you provide a value then it will be used to increment the seed initialization.</font>
#@markdown - <font size="2" color="#5bc0de">For example, if `seed = 1` `batch_size = 10` and `batch_seed = 2` then the seeds produced for each image are `[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]`.</font>

if seeds <= 0:
  seeds = [random.randrange(10, 999999999)]
else:
  seeds = [seeds]
if batch_size > 1:
  if batch_seed <= 0:
    seeds = []
    for s in range(0, batch_size):
      seeds.append(random.randrange(10, 999999999))
  else:
    seeds = [i for i in range(seeds[0] , seeds[0] + (batch_size * batch_seed) - 1, batch_seed)]

width  = 512 #@param {type:"number"}
height = 768 #@param {type:"number"}

#@markdown ---

use_gfpgan = False #@param {type:"boolean"}
#@markdown - <font size="2" color="#5bc0de">**GFPGAN** is used for facial restoration and enhancement and also for Upscaling using **RealERSEGAN**.</font>
gfpgan_model_version = "1.4" #@param ["1", "1.2", "1.3", "1.4"]
x_upscale = 1 # @param {type:"slider", min:1, max:4, step:0.5}

if not flag_gfpgan_installed:
  use_gfpgan = False

# use_prompt_embeddings = True
xnow = datetime.datetime.now()
timt = xnow.strftime("%Y-%m-%d")
im_save = f"/content/drive/MyDrive/SD-IMG-OUT/img/{timt}/"
txt_save = f"/content/drive/MyDrive/SD-IMG-OUT/text/{timt}/"
im_gfpgan = f"/content/GFPGAN/tmp_img/"
os.makedirs(im_save, exist_ok=True)
os.makedirs(txt_save, exist_ok=True)
os.makedirs(im_gfpgan, exist_ok=True)

images = []

for count, seed in enumerate(seeds):
    start_time = time.time()
    if use_prompt_embeddings is False:
        print("Generate "+str(count+1)+" of "+str(len(seeds))+" NOT USE Text Embedding")
        new_img = pipe(
            prompt = prompt,
            negative_prompt = negative_prompt,
            width = width,
            height = height,
            guidance_scale = guidance_scale,
            num_inference_steps = num_inference_steps,
            num_images_per_prompt = 1,
            generator = torch.manual_seed(seed),
        ).images
    else:
        print("Generate "+str(count+1)+" of "+str(len(seeds))+" USE Text Embedding")
        new_img = pipe(
            prompt_embeds = prompt_embeds,
            negative_prompt_embeds = negative_prompt_embeds,
            width = width,
            height = height,
            guidance_scale = guidance_scale,
            num_inference_steps = num_inference_steps,
            num_images_per_prompt = 1,
            generator = torch.manual_seed(seed),
        ).images
    txtsave = prompt+"\n"+"Negative Prompt:"+negative_prompt+"\n"
    txtsave += "steps:"+str(num_inference_steps)+",cfg_scale:"+str(guidance_scale)

    x = datetime.datetime.now()
    tmt = x.strftime("%Y%m%d%H%M%S")
    MID = MODEL_ID.split("/")[-1]
    fname_file = f"{MID}_{tmt}_{seed}"
    with open(f"{txt_save}{fname_file}.txt", "w") as f:
      f.write(txtsave)
    if use_gfpgan:
      new_img[0].save(f"{im_gfpgan}{fname_file}.png")
      # new_img[0].save(f"{im_save}{fname_file}.png")
      new_img[0] = restore_face(im_save, fname_file, gfpgan_model_version, x_upscale)
    else:
      new_img[0].save(f"{im_save}{fname_file}.png")
    images = images + new_img

clear_output()
print("Displaying Images")
gr = image_grid(images)
display(gr)


# 🐛 Debug

In [ ]:
pipe.unload_lora_weights()

In [ ]:
url = "https://drive.google.com/file/d/1C0o6rOzw-EjBh19_a5ZvnJgZ2-fCsw_R/view?usp=drive_link"
output_path = "/content/custom/Stable-diffusion/"
download_drive(url, output_path)
%cd /content/
mod_name = "/content/custom/Stable-diffusion/AZZSFASDL.ckpt"
MODEL_ID = load_model_fromfile(mod_name)

In [ ]:
#@title Batch (Alpha)
#@markdown this not finished yet.
use_prompt_embeddings = True
prompt = """A beautiful woman, detailed, HDR, 8K, high resolution"""

negative_prompt = """deformed, weird, bad resolution, bad depiction,weird, worst quality, worst resolution,too blurry, not relevant"""


prompt_embeds, negative_prompt_embeds = get_prompt_embeddings(
    pipe,
    prompt,
    negative_prompt,
    split_character = ",",
    device = device_name
)

num_inference_steps = 30
guidance_scale = 7

width  = 512
height = 768
use_gfpgan = True
gfpgan_model_version = "1.3"

list_lora = ["/content/custom/Lora/LoRa_V3.safetensors",
             "/content/custom/Lora/LoRa_V2.safetensors",
             "/content/custom/Lora/LoRa_V1.safetensors",
             "/content/custom/Lora/LoRa_V0.safetensors"]
jumlah_generate = 1
images = []
for lc, LORA_PATH in enumerate(list_lora):
  pipe.load_lora_weights(LORA_PATH)
  fnl = LORA_PATH.split("/")[-1]
  fnl = fnl.split(".")[0]
  for count in range(0,jumlah_generate):
    # print(count)
    seed = random.randrange(10, 999999999)
    not_print = "" if use_prompt_embeddings else "NOT"
    print("Lora "+str(lc+1)+" of "+str(len(list_lora))+" Generate - "+str(count+1)+" of "+str(jumlah_generate)+" "+not_print+" USE Text Embedding")
    if use_prompt_embeddings is False:
      new_img = pipe(
            prompt = prompt,
            negative_prompt = negative_prompt,
            width = width,
            height = height,
            guidance_scale = guidance_scale,
            num_inference_steps = num_inference_steps,
            num_images_per_prompt = 1,
            generator = torch.manual_seed(seed),
        ).images
    else:
      new_img = pipe(
            prompt_embeds = prompt_embeds,
            negative_prompt_embeds = negative_prompt_embeds,
            width = width,
            height = height,
            guidance_scale = guidance_scale,
            num_inference_steps = num_inference_steps,
            num_images_per_prompt = 1,
            generator = torch.manual_seed(seed),
        ).images
    txtsave = prompt+"\n"+"Negative Prompt:"+negative_prompt+"\n"
    txtsave += "steps:"+str(num_inference_steps)+",cfg_scale:"+str(guidance_scale)

    x = datetime.datetime.now()
    tmt = x.strftime("%Y%m%d%H%M%S")
    MID = MODEL_ID.split("/")[-1]
    fname_file = f"{fnl}_{tmt}_{seed}"
    with open(f"{txt_save}{fname_file}.txt", "w") as f:
      f.write(txtsave)
    if use_gfpgan:
      new_img[0].save(f"{im_gfpgan}{fname_file}.png")
      # new_img[0].save(f"{im_save}{fname_file}.png")
      new_img[0] = restore_face(im_save, fname_file, gfpgan_model_version)
    else:
      new_img[0].save(f"{im_save}{fname_file}.png")
    images = images + new_img
  pipe.unload_lora_weights()

clear_output()
print("Displaying Images")
gr = image_grid(images)
display(gr)

In [ ]:
obj = os.scandir(im_save)
num = 0
for entry in obj :
  if entry.is_file():
    name = entry.name
    num += 1
print("You have generated "+str(num)+" images.")